In [7]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

# Helper Clases / Functions

In [8]:
def Load_Data(num):
    if (num == -1): # All data
        X_all = []
        y_all = []
        for i in range(8):
            file_path = './../project_datasets/A0' + str(i+1) + 'T_slice.mat'
            data = h5py.File(file_path, 'r')
            X = np.copy(data['image'])
            y = np.copy(data['type'])
            X = X[:, 0:23, :]
            X_all.append(X)
            y = y[0,0:X.shape[0]:1]
            y_all.append(y)
        A, N, E, T = np.shape(X_all)
        X_all = np.reshape(X_all, (A*N, E, T))
        y_all = np.reshape(y_all, (-1))
        y_all = y_all - 769
        ## Remove NAN
        index_Nan = []
        for i in range(A*N):
            for j in range(E):
                if (any(np.isnan(X_all[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X_all = np.delete(X_all, index_Nan, axis=0)
        y_all = np.delete(y_all, index_Nan)
        return (X_all, y_all)
    else:
        file_path = './../project_datasets/A0' + str(num) + 'T_slice.mat'
        data = h5py.File(file_path, 'r')
        X = np.copy(data['image'])
        y = np.copy(data['type'])
        X = X[:, 0:23, :]
        y = y[0,0:X.shape[0]:1]
        y = y - 769
         ## Remove NAN
        N, E, T = np.shape(X)
        index_Nan = []
        for i in range(N):
            for j in range(E):
                if (any(np.isnan(X[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X = np.delete(X, index_Nan, axis=0)
        y = np.delete(y, index_Nan)
        return (X, y)

# Load Data

In [9]:
X, y = Load_Data(-1) # -1 to load all datas
N, E, T = np.shape(X)
print (np.shape(X))

(2280, 23, 1000)


# Make DataLoaders

In [10]:
bs_train = 200
bs_val = 100
bs_test = 100
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dataloaders = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-bs_val-bs_test, bs_val, bs_test])
dataloaders['train'] = data_utils.DataLoader(dset['train'], batch_size=bs_train, shuffle=True, num_workers=1)
dataloaders['val'] = data_utils.DataLoader(dset['val'], batch_size=bs_val, shuffle=True, num_workers=1)
dataloaders['test'] = data_utils.DataLoader(dset['test'], batch_size=bs_test, shuffle=True, num_workers=1)

# Define Model

In [11]:
class myGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myGRU, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        # First Inception layer
        self.conv11 = nn.Conv1d(23, 32, 2, stride=2)
        self.conv12 = nn.Conv1d(23, 32, 4, stride=2, padding=1)
        self.conv13 = nn.Conv1d(23, 32, 8, stride=2, padding=3)
        # Second Inception layer
        self.conv21 = nn.Conv1d(96, 32, 2, stride=2)
        self.conv22 = nn.Conv1d(96, 32, 4, stride=2, padding=1)
        self.conv23 = nn.Conv1d(96, 32, 8, stride=2, padding=3)
        # Third Inception layer
        self.conv31 = nn.Conv1d(96, 32, 2, stride=2)
        self.conv32 = nn.Conv1d(96, 32, 4, stride=2, padding=1)
        self.conv33 = nn.Conv1d(96, 32, 8, stride=2, padding=3)
        #self.conv_13 = nn.Conv2d()
        self.conv_elec = nn.Conv3d(1,23,tuple([40, 23, 1]))
        self.gru1 = nn.GRU(32*3, hidden_dim, num_layer)
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, num_layer)
        self.gru3 = nn.GRU(hidden_dim, hidden_dim, num_layer)
        self.gru4 = nn.GRU(hidden_dim, hidden_dim, num_layer)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        out_conv11 = self.conv11(x)
        out_conv12 = self.conv12(x)
        out_conv13 = self.conv13(x)
        out_conv1 = torch.cat((out_conv11, out_conv12, out_conv13), 1)
        out_conv21 = self.conv21(out_conv1)
        out_conv22 = self.conv22(out_conv1)
        out_conv23 = self.conv23(out_conv1)
        out_conv2 = torch.cat((out_conv21, out_conv22, out_conv23), 1)
        out_conv31 = self.conv31(out_conv2)
        out_conv32 = self.conv32(out_conv2)
        out_conv33 = self.conv33(out_conv2)
        out_conv3 = torch.cat((out_conv31, out_conv32, out_conv33), 1)
        # N, C, L --> L, N, C
        out_conv3 = out_conv3.permute(2,0,1)
        out_gru1, _ = self.gru1(out_conv3)
        out_gru2, _ = self.gru2(out_gru1)
        out_gru3, _ = self.gru3(out_gru2)
        out_gru4, _ = self.gru4(out_gru3)
        out_gru4 = out_gru4[-1, :, :] # taking the last time seq
        out = self.classifier(out_gru4)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

In [15]:
dtype = torch.cuda.FloatTensor
hidden_dim = 32
num_classes = 4
num_epoches = 100
model = myGRU(E, hidden_dim, 1, num_classes)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Conv Training

In [16]:
best_acc = 0.35

In [17]:
for epoch in range(num_epoches):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        X_train, y_train = Variable(X_train), Variable(y_train)
        # forward + backward + optimize
        out = model(X_train.cuda())
        # print (out)
        loss = loss_fn(out, y_train.long().cuda())
        print('(%d batch) loss: %f' % (i, loss))
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = model.check_accuracy(dataloaders['train'])
    val_acc = model.check_accuracy(dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epoches, train_acc, val_acc))
    if (val_acc > best_acc):
        best_acc = val_acc
        torch.save(model, 'best_CHRONET.pt')

(0 batch) loss: 1.406681
(1 batch) loss: 1.394054
(2 batch) loss: 1.383358
(3 batch) loss: 1.388829
(4 batch) loss: 1.390692
(5 batch) loss: 1.393853
(6 batch) loss: 1.392676
(7 batch) loss: 1.381942
(8 batch) loss: 1.389934
(9 batch) loss: 1.386938
(10 batch) loss: 1.392554
(Epoch 1 / 100) train_acc: 0.321154; val_acc: 0.230000
(0 batch) loss: 1.381626
(1 batch) loss: 1.374588
(2 batch) loss: 1.381717
(3 batch) loss: 1.381069
(4 batch) loss: 1.383846
(5 batch) loss: 1.380631
(6 batch) loss: 1.380885
(7 batch) loss: 1.377592
(8 batch) loss: 1.382445
(9 batch) loss: 1.374837
(10 batch) loss: 1.377414
(Epoch 2 / 100) train_acc: 0.323077; val_acc: 0.230000
(0 batch) loss: 1.377000
(1 batch) loss: 1.368068
(2 batch) loss: 1.364079
(3 batch) loss: 1.367991
(4 batch) loss: 1.361467
(5 batch) loss: 1.387448
(6 batch) loss: 1.369591
(7 batch) loss: 1.366379
(8 batch) loss: 1.372883
(9 batch) loss: 1.371547
(10 batch) loss: 1.360708
(Epoch 3 / 100) train_acc: 0.354808; val_acc: 0.280000
(0 batc

/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type myGRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


(0 batch) loss: 1.152367
(1 batch) loss: 1.201174
(2 batch) loss: 1.263725
(3 batch) loss: 1.264408
(4 batch) loss: 1.243412
(5 batch) loss: 1.257376
(6 batch) loss: 1.184273
(7 batch) loss: 1.253968
(8 batch) loss: 1.231770
(9 batch) loss: 1.254390
(10 batch) loss: 1.203172
(Epoch 10 / 100) train_acc: 0.481731; val_acc: 0.240000
(0 batch) loss: 1.190406
(1 batch) loss: 1.174992
(2 batch) loss: 1.109456
(3 batch) loss: 1.204846
(4 batch) loss: 1.248220
(5 batch) loss: 1.214810
(6 batch) loss: 1.187058
(7 batch) loss: 1.223312
(8 batch) loss: 1.147130
(9 batch) loss: 1.212725
(10 batch) loss: 1.264375
(Epoch 11 / 100) train_acc: 0.481250; val_acc: 0.260000
(0 batch) loss: 1.175054
(1 batch) loss: 1.210253
(2 batch) loss: 1.219178
(3 batch) loss: 1.156806
(4 batch) loss: 1.127644
(5 batch) loss: 1.123362
(6 batch) loss: 1.183262
(7 batch) loss: 1.196236
(8 batch) loss: 1.226092
(9 batch) loss: 1.201674
(10 batch) loss: 1.184131
(Epoch 12 / 100) train_acc: 0.542788; val_acc: 0.280000
(0 b

(Epoch 34 / 100) train_acc: 0.664423; val_acc: 0.320000
(0 batch) loss: 0.863215
(1 batch) loss: 0.878012
(2 batch) loss: 0.820602
(3 batch) loss: 0.839150
(4 batch) loss: 0.880896
(5 batch) loss: 0.898408
(6 batch) loss: 0.968772
(7 batch) loss: 0.923268
(8 batch) loss: 0.909196
(9 batch) loss: 0.910115
(10 batch) loss: 1.034934
(Epoch 35 / 100) train_acc: 0.659135; val_acc: 0.330000
(0 batch) loss: 0.916872
(1 batch) loss: 0.807903
(2 batch) loss: 0.813681
(3 batch) loss: 0.950817
(4 batch) loss: 0.907225
(5 batch) loss: 0.788461
(6 batch) loss: 0.941141
(7 batch) loss: 0.909570
(8 batch) loss: 0.894880
(9 batch) loss: 1.022481
(10 batch) loss: 1.062752
(Epoch 36 / 100) train_acc: 0.653846; val_acc: 0.300000
(0 batch) loss: 0.969594
(1 batch) loss: 0.806366
(2 batch) loss: 1.017641
(3 batch) loss: 0.952178
(4 batch) loss: 0.922520
(5 batch) loss: 0.893696
(6 batch) loss: 1.011737
(7 batch) loss: 1.020333
(8 batch) loss: 0.963595
(9 batch) loss: 0.847178
(10 batch) loss: 0.857101
(Epo

(7 batch) loss: 1.000154
(8 batch) loss: 0.899248
(9 batch) loss: 0.933547
(10 batch) loss: 0.875632
(Epoch 59 / 100) train_acc: 0.667788; val_acc: 0.250000
(0 batch) loss: 0.894687
(1 batch) loss: 0.781886
(2 batch) loss: 0.908883
(3 batch) loss: 0.936447
(4 batch) loss: 0.858201
(5 batch) loss: 0.937011
(6 batch) loss: 0.897885
(7 batch) loss: 1.070326
(8 batch) loss: 0.911178
(9 batch) loss: 0.937916
(10 batch) loss: 0.882809
(Epoch 60 / 100) train_acc: 0.646154; val_acc: 0.230000
(0 batch) loss: 0.802470
(1 batch) loss: 0.831645
(2 batch) loss: 0.866271
(3 batch) loss: 0.960139
(4 batch) loss: 0.881583
(5 batch) loss: 0.929146
(6 batch) loss: 0.950199
(7 batch) loss: 0.901900
(8 batch) loss: 0.965763
(9 batch) loss: 1.003978
(10 batch) loss: 1.004749
(Epoch 61 / 100) train_acc: 0.629327; val_acc: 0.270000
(0 batch) loss: 0.919446
(1 batch) loss: 0.849333
(2 batch) loss: 0.871869
(3 batch) loss: 0.879656
(4 batch) loss: 0.936000
(5 batch) loss: 0.948588
(6 batch) loss: 0.995162
(7 b

Process Process-248:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


(7 batch) loss: 0.846874


Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7fbb0499f630>>
Traceback (most recent call last):
  File "/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self._shutdown_workers()
  File "/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
    self.data_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 345, in get
    return ForkingPickler.loads(res)
  File "/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", l

KeyboardInterrupt: 

# Best Model Test

In [ ]:
best_model = torch.load('best_CHRONET.pt')
print (best_model.check_accuracy(dataloaders['val']))